![](./resources/System_v1_inference_header.png)

This notebook contains a short demo on how to use the WorldCereal system to generate a cropland extent and crop type map,
using default models trained by the WorldCereal consortium.

# Content

- [Before you start](#before-you-start)
- [1. Define region of interest](#1.-Define-a-region-of-interest)
- [2. Generate default Worldcereal products](#2.-Generate-default-WorldCereal-products)
- [3. Explore available reference data](#3.-Explore-available-reference-data)

# Before you start

In order to run this notebook, you need to create an account on the Copernicus Data Space Ecosystem,
by completing the form [HERE](https://identity.dataspace.copernicus.eu/auth/realms/CDSE/login-actions/registration?client_id=cdse-public&tab_id=eRKGqDvoYI0).


In [1]:
# First we import the necessary modules to run this notebook

import pandas as pd
import geopandas as gpd
import requests
from pathlib import Path
from shapely.geometry import shape, Polygon
import xarray as xr

import openeo
from openeo_gfmap import BoundingBoxExtent, TemporalContext
from openeo_gfmap.backend import Backend, BackendContext

import sys
sys.path.append('/home/jovyan/worldcereal-classification/src')

from worldcereal.utils.map import (get_ui_map, _latlon_to_utm)
from worldcereal.utils.refdata import _to_points
from worldcereal.utils.wrapper import run_inference

RDM_API = 'https://ewoc-rdm-api.iiasa.ac.at'

# 1. Define a region of interest

When running the code snippet below, an interactive map will be visualized.
Click the Rectangle button on the left hand side of the map to start drawing your region of interest.
When finished, execute the second cell to store the coordinates of your region of interest. 

In [2]:
m, dc = get_ui_map()
m

Map(center=[51.1872, 5.1154], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoo…

In [3]:
# retrieve bounding box from drawn rectangle
obj = dc.last_draw
if obj.get('geometry') is not None:
    poly = Polygon(shape(obj.get('geometry')))
    bbox = poly.bounds
else:
    raise ValueError('Please first draw a rectangle '
                     'on the map before proceeding.')
print(f'Your area of interest: {bbox}')

Your area of interest: (4.182795, 51.275179, 4.217471, 51.294719)


# 2. Generate default WorldCereal products

In this section we will generate a WorldCereal mapping product for our region of interest using a series of pre-trained models made available by the consortium. Aside from our region, we will need to specify the type of product to generate and the time frame (year or agricultural season) of interest.

In this example, we will generate a temporary crops product, followed by a crop type product for maize.

We start with the temporary crops product, which is by default generated for a full calendar year:

In [4]:
#TODO: use crop calendars to determine start and end date

#TODO: switch to GTiff as output format (for now I manually converted the resulting netcdf to GTiff)


# Defining period of interest for temporary crops product
start_date = '2021-01-01'
end_date = '2021-12-31'
temporal_extent = TemporalContext(start_date, end_date)

# We convert our bounding box to local UTM projection
bbox_utm, epsg = _latlon_to_utm(bbox)
spatial_extent = BoundingBoxExtent(bbox_utm[0], bbox_utm[1],
                                   bbox_utm[2], bbox_utm[3], epsg)

# We specify the cloud backend where computations need to be done
backend_context = BackendContext(Backend.FED)

# Specify path where to store the results and create this folder
# output_path = '/home/joyvan/test/temporal_crops.nc'
output_path = '/vitodata/worldcereal/test/temporal_crops.tif'
Path(output_path).parent.mkdir(parents=True, exist_ok=True)

outfile = run_inference(spatial_extent, temporal_extent, backend_context,
                        output_path, product='cropland', format='NetCDF')


Authenticated using refresh token.


Selected orbit direction: ASCENDING from max accumulated area overlap between bounds and products.
0:00:00 Job 'j-24061812882742d58706ceaad3c1b55e': send 'start'
0:00:23 Job 'j-24061812882742d58706ceaad3c1b55e': created (progress 0%)
0:00:29 Job 'j-24061812882742d58706ceaad3c1b55e': created (progress 0%)
0:00:35 Job 'j-24061812882742d58706ceaad3c1b55e': created (progress 0%)
0:00:44 Job 'j-24061812882742d58706ceaad3c1b55e': running (progress N/A)
0:00:54 Job 'j-24061812882742d58706ceaad3c1b55e': running (progress N/A)
0:01:06 Job 'j-24061812882742d58706ceaad3c1b55e': running (progress N/A)
0:01:23 Job 'j-24061812882742d58706ceaad3c1b55e': running (progress N/A)
0:01:42 Job 'j-24061812882742d58706ceaad3c1b55e': running (progress N/A)
0:02:07 Job 'j-24061812882742d58706ceaad3c1b55e': running (progress N/A)
0:02:37 Job 'j-24061812882742d58706ceaad3c1b55e': running (progress N/A)
0:03:15 Job 'j-24061812882742d58706ceaad3c1b55e': running (progress N/A)
0:04:02 Job 'j-24061812882742d58706cea

INFO:openeo.rest.job:Downloading Job result asset 'openEO.nc' from https://openeo.creo.vito.be/openeo/jobs/j-24061812882742d58706ceaad3c1b55e/results/assets/ZGNjYWI2ZDktODQ2Yy00OGE5LTlkOTQtNDk3MTQ2Y2IyMjg1/eab4d18548dc8c1c66040d33a57b8d82/openEO.nc?expires=1719319845 to /vitodata/worldcereal/test/temporal_crops.nc


### IMPORTANT INFORMATION TO ENSURE DISPLAY OF RESULTS WORKS PROPERLY

if working on binder, set localtileserver client prefix
import os
os.environ['LOCALTILESERVER_CLIENT_PREFIX'] = "proxy/{{port}}"

if working on terrascope virtual machine, ensure that you forward the port of the localtileserver
1) in the add_raster function, add the following argument: port=LOCALTILESERVER_PORT
2) ensure that whichever number you defined as the LOCALTILESERVER_PORT, this port is forwarded to your local machine
e.g. Port 7777, Forwarded address: localhost:7778


In [1]:
LOCALTILESERVER_PORT = 7778

import leafmap

m = leafmap.Map()

# Visualize the cropland extent product using leafmap
outfile = '/vitodata/worldcereal/test/temporal_crops.tif'
m.add_raster(outfile, layer_name="cropland", opacity=0.7,
             port=LOCALTILESERVER_PORT)
m

Map(center=[20, 0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_text…

# 3. Explore available reference data

In-situ reference data used for training and validating WorldCereal classification models is hosted in the WorldCereal Reference Data Module (RDM). This module can be accessed through a dedicated user interface (https://ewoc-rdm-ui.iiasa.ac.at/map), but also through an API, which will be demonstrated here.

In this section we will explore the available reference datasets for our region of interest. In case you would like to test multiple regions of interest, execute the first two cells in [this section](#1.-Define-a-region-of-interest) of the notebook.

In [4]:
# Check full list of available collections
headers = {}
collectionResponse = requests.get(f'{RDM_API}/collections', headers=headers)
collections = collectionResponse.json()
col_ids = [x['collectionId'] for x in collections['items']]
print(f'Available collections: {col_ids}')

# Now we check which collections intersect with our AOI
bbox_str = f'Bbox={bbox[0]}&Bbox={bbox[1]}&Bbox={bbox[2]}&Bbox={bbox[3]}'
colSearchUrl = f'{RDM_API}/collections/search?{bbox_str}'
colSearchResponse = requests.get(colSearchUrl, headers=headers)
test = colSearchResponse.json()
print('The following collections intersect with your AOI:')
for i, col in enumerate(test):
    print()
    print(f'Collection {i+1}: {col["collectionId"]}')


Available collections: ['2017afoneacrefundmelpoint110', '2017ascawaprojectpoly111', '2017bellpisflanderspoly110', '2017bfajecamciradpoly111', '2017brajecamciradpoly111', '2017canaafccropinventorypoint110', '2017cmrcgiargardianpoint110', '2017lbnfaowapor1poly111', '2017lbnfaowapor2poly111', '2017mdgjecamciradpoly111']
The following collections intersect with your AOI:

Collection 1: 2017bellpisflanderspoly110

Collection 2: 2018eulucas2018point110

Collection 3: 2019beflandersfullpoly110

Collection 4: 2020nleurocroppoly110


In [ ]:
# Now that our reference data has been added,
# we launch a request to extract all reference points and polygons
# intersecting our region of interest.
# --> this should return the selected points/polygons as a geojson file/object
# (one per collection).

# To combine the data into one database, we immediately convert
# the polygons to points (by taking the centroid).

max_items = 2000

dfs = []
for col in cols_aoi:
    itemSearchCollectionId = col['collectionId']
    print(
        f'Extracting reference data from collection {itemSearchCollectionId}')
    itemSearchUrl = f'{RDM_API}/collections/{itemSearchCollectionId}/items?{bbox_str}&MaxResultCount={max_items}'
    itemSearchResponse = requests.get(itemSearchUrl, headers=headers)
    df = gpd.GeoDataFrame.from_features(
        itemSearchResponse.json(), crs='EPSG:4326')
    dfs.append(_to_points(df))

gdf = pd.concat(dfs, ignore_index=True)
print(f'Got a total of {len(gdf)} reference points')